# FHE Aging

## Setup

In [1]:
import pandas as pd 
import pyaging as pya

pya.data.download_example_data('blood_chemistry_example')
df = pd.read_pickle('pyaging_data/blood_chemistry_example.pkl')
adata = pya.preprocess.df_to_adata(df)

|-----> 🏗️ Starting download_example_data function
|-----------> Data found in pyaging_data/blood_chemistry_example.pkl
|-----> 🎉 Done! [0.0036s]
|-----> 🏗️ Starting df_to_adata function
|-----> ⚙️ Create anndata object started
|-----> ✅ Create anndata object finished [0.0121s]
|-----> ⚙️ Add metadata to anndata started
|-----------? No metadata provided. Leaving adata.obs empty
|-----> ⚠️ Add metadata to anndata finished [0.0255s]
|-----> ⚙️ Log data statistics started
|-----------> There are 30 observations
|-----------> There are 10 features
|-----------> Total missing values: 0
|-----------> Percentage of missing values: 0.00%
|-----> ✅ Log data statistics finished [0.0575s]
|-----> ⚙️ Impute missing values started
|-----------> No missing values found. No imputation necessary
|-----> ✅ Impute missing values finished [0.0050s]
|-----> 🎉 Done! [0.1781s]


In [2]:
import marshal
import math
import ntpath
import os
import types
from typing import Dict, List, Tuple
from urllib.request import urlretrieve

import anndata
import numpy as np
import pandas as pd
import torch
from anndata.experimental.pytorch import AnnLoader
from torch.utils.data import DataLoader, TensorDataset

from pyaging.logger import LoggerManager, main_tqdm, silence_logger
from pyaging.models import *
from pyaging.utils import download, load_clock_metadata, progress
from pyaging.predict._postprocessing import *
from pyaging.predict._preprocessing import *

@progress("Predict ages with model")
def predict_ages_with_model(
    adata: anndata.AnnData,
    model: pyagingModel,
    device: str,
    batch_size: int,
    logger,
    indent_level: int = 2,
) -> torch.Tensor:
    """
    Predict biological ages using a trained model and input data.

    This function takes a machine learning model and input data, and returns predictions made by the model.
    It's primarily used for estimating biological ages based on various biological markers. The function
    assumes that the model is already trained. A dataloader is used because of possible memory constraints
    for large datasets.

    Parameters
    ----------
    adata : anndata.AnnData
        The AnnData object containing the dataset. Its `.X` attribute is expected to be a matrix where rows
        correspond to samples and columns correspond to features.

    model : pyagingModel
        The pyagingModel of the aging clock of interest.

    device : str
        Device to move AnnData to during inference. Eithe 'cpu' or 'cuda'.

    batch_size : int
        Batch size for the AnnLoader object to predict age.

    logger : Logger
        A logger object for logging the progress or any relevant information during the prediction process.

    indent_level : int, optional
        The indentation level for logging messages, by default 2.

    Returns
    -------
    predictions : torch.Tensor
        An array of predicted ages or biological markers, as returned by the model.

    Notes
    -----
    Ensure that the data is preprocessed (e.g., scaled, normalized) as required by the model before
    passing it to this function. The model should be in evaluation mode if it's a type that has different
    behavior during training and inference (e.g., PyTorch models).

    The exact nature of the predictions (e.g., age, biological markers) depends on the model being used.

    Examples
    --------
    >>> model = load_pretrained_model()
    >>> predictions = predict_ages_with_model(model, "cpu", logger)
    >>> print(predictions[:5])
    [34.5, 29.3, 47.8, 50.1, 42.6]

    """

    # If there is a preprocessing step
    if model.preprocess_name is not None:
        logger.info(
            f"The preprocessing method is {model.preprocess_name}",
            indent_level=indent_level + 1,
        )
    else:
        logger.info("There is no preprocessing necessary", indent_level=indent_level + 1)

    # If there is a postprocessing step
    if model.postprocess_name is not None:
        logger.info(
            f"The postprocessing method is {model.postprocess_name}",
            indent_level=indent_level + 1,
        )
    else:
        logger.info("There is no postprocessing necessary", indent_level=indent_level + 1)

    # Create an AnnLoader
    use_cuda = torch.cuda.is_available()
    dataloader = AnnLoader(adata, batch_size=batch_size, use_cuda=use_cuda)

    # with torch.no_grad():
    #     for param in model.parameters():
    #         param.zero_()

    # Use the AnnLoader for batched prediction
    predictions = []
    with torch.inference_mode():
        for batch in main_tqdm(dataloader, indent_level=indent_level + 1, logger=logger):
            batch_pred = model(batch.obsm[f"X_{model.metadata['clock_name']}"])
            predictions.append(batch_pred)
    # Concatenate all batch predictions
    predictions = torch.cat(predictions)

    return predictions

pya.pred.predict_age_fhe(adata, predict_ages_with_model, 'PhenoAge')

|-----> 🏗️ Starting predict_age function
|-----> ⚙️ Set PyTorch device started
|-----------> Using device: cpu
|-----> ✅ Set PyTorch device finished [0.0011s]
|-----> 🕒 Processing clock: phenoage
|-----------> ⚙️ Load clock started
|-----------------> Data found in pyaging_data/phenoage.pt
Layer: base_model.linear.weight | Size: torch.Size([1, 10]) | Values : tensor([[-0.0336,  0.0095,  0.1953,  0.0954, -0.0120,  0.0268,  0.3306,  0.0019,
          0.0554,  0.0804]], dtype=torch.float64, grad_fn=<SliceBackward0>) 

Layer: base_model.linear.bias | Size: torch.Size([1]) | Values : tensor([-19.9067], dtype=torch.float64, grad_fn=<SliceBackward0>) 

|-----------> ✅ Load clock finished [0.0560s]
|-----------> ⚙️ Check features in adata started
|-----------------> All features are present in adata.var_names.
|-----------> ✅ Check features in adata finished [0.0014s]
|-----------> ⚙️ Predict ages with model started
|-----------------> There is no preprocessing necessary
|-----------------> Th

In [3]:
combined_df = pd.DataFrame({
    'phenoage': adata.obs["phenoage"],
    'chronological_age': df["age"]
})

# Display the first few rows
combined_df[:15]

,phenoage,chronological_age
patient1,74.348798,70.2
patient2,67.372000,76.5
patient3,74.789739,66.4
patient4,46.991769,46.5
patient5,44.559486,42.3
patient6,72.509460,76.9
patient7,57.377050,55.1
patient8,31.779798,34.6
patient9,50.356509,47.3
patient10,67.696706,52.3


## Manual Torch Execution

In [5]:
class PhenoAge(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        return x

    def postprocess(self, x):
        """
        Applies a convertion from a CDF of the mortality score from a Gompertz
        distribution to phenotypic age.
        """
        # lambda
        l = torch.tensor(0.0192, device=x.device, dtype=x.dtype)
        mortality_score = 1 - torch.exp(-torch.exp(x) * (torch.exp(120 * l) - 1) / l)
        age = 141.50225 + torch.log(-0.00553 * torch.log(1 - mortality_score)) / 0.090165
        return age

In [6]:
device = "cpu"
weights_path = f"./pyaging_data/phenoage.pt"
clock = torch.load(weights_path, weights_only=False)

for name, param in clock.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

# Prepare clock for inference
clock.to(torch.float64)
clock.to(device)
clock.eval()

Layer: base_model.linear.weight | Size: torch.Size([1, 10]) | Values : tensor([[-0.0336,  0.0095,  0.1953,  0.0954, -0.0120,  0.0268,  0.3306,  0.0019,
          0.0554,  0.0804]], dtype=torch.float64, grad_fn=<SliceBackward0>) 

Layer: base_model.linear.bias | Size: torch.Size([1]) | Values : tensor([-19.9067], dtype=torch.float64, grad_fn=<SliceBackward0>) 



PhenoAge(
  (base_model): LinearModel(
    (linear): Linear(in_features=10, out_features=1, bias=True)
  )
)

In [7]:
adata

AnnData object with n_obs × n_vars = 30 × 10
    obs: 'phenoage'
    var: 'percent_na'
    uns: 'phenoage_percent_na', 'phenoage_missing_features', 'phenoage_metadata'
    layers: 'X_original'

In [8]:
clock.metadata['clock_name']

'phenoage'

In [9]:
adata.obs

,phenoage
patient1,74.348798
patient2,67.372000
patient3,74.789739
patient4,46.991769
patient5,44.559486
patient6,72.509460
patient7,57.377050
patient8,31.779798
patient9,50.356509
patient10,67.696706


In [10]:
adata.obs.to_csv("out.csv")

In [11]:
dataset = df.iloc[:, 0:10]

In [12]:
dataset_np = dataset.to_numpy()
phenoages_np = np.array(adata.obs["phenoage"], dtype=np.float64)

In [13]:
dataset_torch = torch.tensor(dataset_np, dtype=torch.float64)
phenoages_torch = torch.tensor(phenoages_np, dtype=torch.float64)

In [14]:
with torch.inference_mode():
    pred = clock(dataset_torch)

In [15]:
pred

tensor([[74.3488],
        [67.3720],
        [74.7897],
        [46.9918],
        [44.5595],
        [72.5095],
        [57.3771],
        [31.7798],
        [50.3565],
        [67.6967],
        [62.6020],
        [41.7359],
        [82.2387],
        [56.6775],
        [46.4021],
        [63.7108],
        [84.7842],
        [87.1650],
        [90.2054],
        [62.2351],
        [25.2728],
        [55.2115],
        [69.7079],
        [49.1802],
        [45.2600],
        [35.3339],
        [81.8737],
        [64.5594],
        [79.2270],
        [58.7839]], dtype=torch.float64)

## PhenoAge Model (FHE)

### Port Model to SKLearn

In [16]:
import numpy as np
from sklearn.linear_model import ElasticNet

manual_coefficients = np.array([
    -0.0336,  0.0095,  0.1953,  0.0954, -0.0120,  0.0268,  0.3306,  0.0019, 0.0554,  0.0804
])
manual_intercept = np.array([
    -19.9067
])
sklearn_model = ElasticNet()
sklearn_model.n_features_in_ = len(manual_coefficients)
sklearn_model.coef_ = manual_coefficients
sklearn_model.intercept_ = manual_intercept

raw_pred = sklearn_model.predict(dataset_np)
pred_torch = torch.tensor(raw_pred, dtype=torch.float32)

phenoage_post = PhenoAge()
pred = phenoage_post.postprocess(pred_torch)

combined_df["skelearn_phenoage_pred"] = pred

In [17]:
from concrete.ml.sklearn import ElasticNet
cml_model = ElasticNet.from_sklearn_model(
    sklearn_model, dataset_np, n_bits=16)

# Compile the model:
cml_model.compile(dataset_np)

y_pred_fhe = cml_model.predict(dataset_np, fhe="execute")

fhe_phenoages = phenoage_post.postprocess(torch.tensor(y_pred_fhe))
combined_df["fhe_phenoage_pred"] = fhe_phenoages
fhe_phenoages

tensor([74.3435, 67.3695, 74.7869, 46.9877, 44.5540, 72.5062, 57.3715, 31.7737,
        50.3477, 67.6901, 62.6009, 41.7315, 82.2335, 56.6703, 46.3981, 63.7043,
        84.7760, 87.1570, 90.2006, 62.2321, 25.2676, 55.2072, 69.7042, 49.1742,
        45.2576, 35.3276, 81.8721, 64.5565, 79.2244, 58.7759],
       dtype=torch.float64)

In [18]:
fhe_vs_pheno_pred_error = combined_df.apply(lambda row: abs(row["phenoage"] - row["fhe_phenoage_pred"]), axis=1)
combined_df["fhe_vs_phenoage"] = fhe_vs_pheno_pred_error
fhe_mae = sum(combined_df["fhe_vs_phenoage"]) / combined_df["fhe_vs_phenoage"].shape[0]
fhe_mae

0.004873293246623689

In [19]:
combined_df

,phenoage,chronological_age,skelearn_phenoage_pred,fhe_phenoage_pred,fhe_vs_phenoage
patient1,74.348798,70.2,74.348793,74.343519,0.005279
patient2,67.372000,76.5,67.372002,67.369508,0.002492
patient3,74.789739,66.4,74.789734,74.786881,0.002858
patient4,46.991769,46.5,46.991768,46.987679,0.004090
patient5,44.559486,42.3,44.559494,44.554036,0.005449
patient6,72.509460,76.9,72.509460,72.506231,0.003229
patient7,57.377050,55.1,57.377045,57.371516,0.005534
patient8,31.779798,34.6,31.779800,31.773703,0.006095
patient9,50.356509,47.3,50.356499,50.347676,0.008833
patient10,67.696706,52.3,67.696709,67.690051,0.006655
